# 2. Texto com Deep Belief Networks

### a) Dataset

O conjunto de dados 20 Newsgroups é uma coleção de aproximadamente 20.000 documentos de grupo de notícias, particionado (quase) uniformemente em 20 groups diferentes. Para o melhor de meu conhecimento, ele foi originalmente coletado para o seu Newsweeder: Uma rede que aprender a filtrar noticias. A coleção de 20 newsgroups tornou-se um popular conjunto de dados para experiências em aplicações de texto de técnicas de aprendizagem de máquinas, como classificação de texto e agrupamento de texto.

#### Organização

Os dados estão organizados em 20 diferentes newsgroups, cada um correspondente a um tópico diferente. Alguns dos newsgroups estão intimamente relacionados uns com os outros (por exemplo, comp.sys.ibm.pc.hardware / comp.sys.mac.hardware), enquanto outros são altamente não relacionados (por exemplo, misc.forsale / soc.religion.christian). Aqui está uma lista dos 20 newsgroups, particionados (mais ou menos) de acordo com o assunto:

![alt text](img/text.png)

#### Importamos os modulos de Python necessarios para este exemplo:

In [ ]:
import os
import sys
import timeit

import numpy

import theano
import theano.tensor as T
from theano.sandbox.rng_mrg import MRG_RandomStreams

from DBN import DBN , news20group_load_data

### a) Configurar e compilar o modelo Deep Belief Network. 

Configuração de parametros para pre-treinamento e sintonização

```python
# Parametros de pre-treinamento e treinamento
finetune_lr=0.1
pretraining_epochs=100#100
pretrain_lr=0.01
k=3
training_epochs=1000
dataset='20newsgroup.mat'
batch_size=10

# Abrir os dados
datasets = news20group_load_data(dataset)
train_set_x, train_set_y = datasets[0]
test_set_x, test_set_y = datasets[1]

# Calcular o número de minibatches para treinamento
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size
```

In [ ]:
# Parametros de pre-treinamento e treinamento
finetune_lr=0.1
pretraining_epochs=20#100
pretrain_lr=0.1
k=1
training_epochs=10
dataset='20newsgroups_small.mat'#'20newsgroup.mat' #
batch_size=10

# Abrir os dados
datasets = news20group_load_data(dataset)
train_set_x, train_set_y = datasets[0]
test_set_x, test_set_y = datasets[1]

# Calcular o número de minibatches para treinamento
n_train_batches = train_set_x.get_value(borrow=True).shape[0] // batch_size

Visualizamos a distribuição dos dados:

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

y=train_set_y.eval()
plt.hist(y)

Compilamos o modelo:

In [ ]:
# numpy random generator
numpy_rng = numpy.random.RandomState(152)
hidden_layers_sizes = [500,500,2000]

print('... building the model')
# construct the Deep Belief Network
dbn = DBN(numpy_rng=numpy_rng, n_ins=5000,
          hidden_layers_sizes = hidden_layers_sizes,
          n_outs=20)

# Pre-train layer-wise

```python
print('... getting the pretraining functions')
pretraining_fns = dbn.pretraining_functions(train_set_x=train_set_x,
                                            batch_size=batch_size,
                                            k=k)

print('... pre-training the model')
start_time = timeit.default_timer()

for i in range(dbn.n_layers):
    # go through pretraining epochs
    for epoch in range(pretraining_epochs):
        # go through the training set
        c = []
        for batch_index in range(n_train_batches):
            c.append(pretraining_fns[i](index=batch_index,
                                        lr=pretrain_lr))
        print('Pre-training layer %i, epoch %d, cost ' % (i, epoch), end=' ')
        print(numpy.mean(c, dtype='float64'))

end_time = timeit.default_timer()
print('The pretraining code, ' +  'ran for %.2fm' % ((end_time - start_time) / 60.), file=sys.stderr)
```
# end


### c) Sintonizar o modelo

In [ ]:
dbn.train(datasets = datasets , 
          batch_size = batch_size, 
          finetune_lr = finetune_lr, 
          n_train_batches = n_train_batches,
          training_epochs = training_epochs)

### d) Extraer os features pra ser logo analizados

Criamos um preditor, que permite retornar as saidas nas diferentes camadas no modelo.

In [ ]:
predict = dbn.build_predict_functions(dataset_x=test_set_x, batch_size=batch_size)

Extraemos os features para os dados de teste:

In [ ]:
len_data = test_set_x.eval().shape[0]

num_iter = int(len_data / batch_size)

features_1 = numpy.zeros((len_data, hidden_layers_sizes[0]))
features_2 = numpy.zeros((len_data, hidden_layers_sizes[1]))
features_3 = numpy.zeros((len_data, hidden_layers_sizes[2]))
features_4 = numpy.zeros((len_data, 20))

for i in range(num_iter):
    f1,f2,f3,f4 = tuple(predict(i))
    features_1[i*batch_size: (i + 1) * batch_size] = f1
    features_2[i*batch_size: (i + 1) * batch_size] = f2
    features_3[i*batch_size: (i + 1) * batch_size] = f3
    features_4[i*batch_size: (i + 1) * batch_size] = f4

Salvamos os features de uma camada especifica para analizar em Tensorboard:

In [ ]:
import pickle
pickle.dump( [features_4, test_set_y.eval()], open( "features.p", "wb" ) )